#### Task 8.1 Create a bubble map for property crimes in Boston, 2019. Each bubble represents one property crime incident. Your map should look something like this example (Links to an external site.) online. ####

In [2]:
# import packages and cd
import geopandas as gpd
import pandas as pd
import os
import seaborn as sns
import folium
import random


from folium import Choropleth, Circle, Marker
from folium.plugins import HeatMap, MarkerCluster


os.chdir('/Users/[editted]/Documents/compsoc/code/boston_crime2019')

# use cleaned 2019 crime data and create variables of interest
cleaned_crime_df=pd.read_csv('df_clean.csv',low_memory=False)
cleaned_crime_df.head() 

cleaned_crime_df['offense_type'] = None
cleaned_crime_df['offense_type'][cleaned_crime_df['OFFENSE_DESCRIPTION'].str.lower().str.contains('robbery|assault|rape')] = 'violent crime'
cleaned_crime_df['offense_type'][cleaned_crime_df['OFFENSE_DESCRIPTION'].str.lower().str.contains('burglary|larceny|theft|arson')] = 'property crime'

cleaned_crime_df

#construct geometry from lat/lon and convert pandas to geopandas
gpd.points_from_xy(cleaned_crime_df['Long'], cleaned_crime_df['Lat'])

crime_gdf = gpd.GeoDataFrame(cleaned_crime_df, geometry=gpd.points_from_xy(cleaned_crime_df['Long'], cleaned_crime_df['Lat']))

crime_gdf.head()

#keep only property crime 2019 data 
propcrime_gdf=crime_gdf.loc[crime_gdf['offense_type']=='property crime']
propcrime_gdf=propcrime_gdf.loc[propcrime_gdf['YEAR']==2019]

print(propcrime_gdf.shape) #data size from 151895 to 12897
propcrime_gdf



FileNotFoundError: [Errno 2] No such file or directory: '/Users/[editted]/Documents/compsoc/code/boston_crime2019'

In [ ]:
#map 

propcrime_gdf = propcrime_gdf[propcrime_gdf['Long'] < 0].reset_index()

clon, clat = propcrime_gdf['Long'].mean(), propcrime_gdf['Lat'].mean()
m = folium.Map(location=(clat, clon), zoom_start=12, width=800, height=800)


for i in range(len(propcrime_gdf)):
    crime_long, crime_lat = propcrime_gdf.loc[i, 'Long'], propcrime_gdf.loc[i, 'Lat']
    folium.CircleMarker((crime_lat, crime_long), opacity=0.2, color='red',fill_color='red', weight=0.2, radius=0.1).add_to(m)

display(m)


#### Task 8.2 Create another bubble map for violent crimes in Boston, 2019. Use a different map design (marker color, tile style, etc). ####

In [ ]:
# keep only 2019 violent crime data point
viocrime_gdf=crime_gdf.loc[crime_gdf['offense_type']=='violent crime']
viocrime_gdf=viocrime_gdf[viocrime_gdf['YEAR']==2019]

print(viocrime_gdf.shape) #data size from 151895 to 8002
viocrime_gdf


In [ ]:
# map
viocrime_gdf = viocrime_gdf[viocrime_gdf['Long'] < 0].reset_index()

clon, clat = viocrime_gdf['Long'].mean(), viocrime_gdf['Lat'].mean()
map2 = folium.Map(location=(clat, clon), zoom_start=12, width=800, height=400, tiles="Cartodb Positron")

for i in range(len(viocrime_gdf)):
    crime_long, crime_lat = viocrime_gdf.loc[i, 'Long'], viocrime_gdf.loc[i, 'Lat']
    folium.CircleMarker((crime_lat, crime_long), opacity=0.2, color='blue',fill_color='blue', weight=0.2, radius=0.1).add_to(map2)


In [ ]:
map2

#### Task 8.3 Calculate zoning district-level violent vs. property crime counts. Use boston's zoning map from here (Links to an external site.) to map individual crime incidents to zoning district. Create a bubble map for these two sets of counts. There should be two sets of bubbles in your map. One set represents district-level violent crime counts, and the other set district-level property crime counts. These two bubble sets should be colored differently, but the size of the bubble for both sets should indicate the relative size of the crime count. Use each district’s centroid location to place their bubbles on the map. [Hint: in geopandas,  `gdf[“geometry”].centroid` returns the centroid point of the corresponding geometry.] #### 


In [ ]:
# read boston district data
boston_gdf = gpd.read_file('Boston_Zoning_Subdistricts.geojson')
print(boston_gdf.shape)
print(boston_gdf.columns)
boston_gdf.head()

# spatial join: point within polygon
crime_with_zone_gdf = gpd.sjoin(crime_gdf, boston_gdf, predicate="within", how='left')

crime_with_zone_gdf=crime_with_zone_gdf[crime_with_zone_gdf['YEAR']==2019] #keep only 2019 data

# check basic data characteristcs 
crime_with_zone_gdf.head()
print(crime_with_zone_gdf.columns)
print(crime_with_zone_gdf.shape)

# Calculate zoning district-level violent vs. property crime counts
crime_by_district = crime_with_zone_gdf.groupby(['offense_type', 'OBJECTID']).agg({'INCIDENT_NUMBER': 'count'}).reset_index()
crime_by_district['OBJECTID'] = crime_by_district['OBJECTID'].astype(int)
crime_by_district

crime_count_gdf = boston_gdf[['OBJECTID', 'geometry']].merge(crime_by_district, how='inner', on='OBJECTID')


crime_count_points = crime_count_gdf.copy()
crime_count_points['geometry'] = crime_count_points['geometry'].centroid


In [ ]:
import plotly.express as px
import matplotlib.pyplot as plt

crime_count_points_4326 = crime_count_points.to_crs("EPSG:4326")
fig = px.scatter_mapbox(crime_count_points_4326, 
                        lat=crime_count_points_4326.geometry.y, 
                        lon=crime_count_points_4326.geometry.x, 
                        size="INCIDENT_NUMBER", 
                        color="offense_type", 
                        hover_name = "INCIDENT_NUMBER",
                  color_continuous_scale=px.colors.colorbrewer.Reds,  
                        size_max=15, 
                        zoom=10
                    )
fig.update_layout(mapbox_style="carto-positron")
# fig.show()



# alternative way
# fig, ax = plt.subplots(figsize=(16,16))
# crime_count_gdf.plot(ax=ax, color="lightgray", edgecolor="grey", linewidth=0.4)
# crime_count_points.plot(ax=ax,color="red", markersize="INCIDENT_NUMBER",alpha=0.7, categorical=False, legend=True)
# ax.axis("off")
# plt.axis('equal')
# plt.show()
